# Implementing Quantum One-time pad with Cirq

This notebook is based on [lecture notes from TU Delft university](https://ocw.tudelft.nl/wp-content/uploads/LN_Week1.pdf)

Today we will see simple quantum encryption technique which is analogous to a one-time pad.

## What is _classical_ one-time pad?

First, let us understand and implement the idea behind the classical algorithm. We will consider the following scenario - Alice and Bob want to share a secret message between them. The one-time pad is a technique that will take this message, together with a _secret key_ and produce an encrypted message.

The _secret key_ is simply a binary string of the same length as the secret message. What we've done here is basically "abstracted out" the problem of encoding the message to a provider of the secret key. To ensure that the eavesdropper cannot learn the content of the message, we must have secret keys that are only known to Alice and Bob.

But - technically the idea is simple, we will take a binary string message and add (mod 2)/XOR the secret key to create encrypted message

In [46]:
import numpy as np

secret_key_known_only_to_alice_and_bob = np.array([1, 1, 1, 0, 1, 0, 1, 0])
alice_message = np.array([1, 0, 1, 0, 1, 1, 0, 1])

encrypted_message = alice_message ^ secret_key_known_only_to_alice_and_bob # XOR operation
print(encrypted_message)

[0 1 0 0 0 1 1 1]


Of course now we need a way for Bob to decrypt the message. The interesting fact about XOR operation is that if we apply it twice, we will end up with original string, so `(a ^ b ^ b == a)`. By our assumption Alice and Bob fully securely share `secret_key_known_only_to_alice_and_bob`. We hence can do the following.

In [47]:
bob_decrypted_message = encrypted_message ^ secret_key_known_only_to_alice_and_bob
print("Bob decoded message:", bob_decrypted_message, "Is Bob and Alice message equal?", np.all(alice_message == bob_decrypted_message))

Bob decoded message: [1 0 1 0 1 1 0 1] Is Bob and Alice message equal? True


## Jumping into quantum world

As we can see the algorithm is straight forward. We still need to assume that Alice and Bob shared the secret keys beforehand. First let's try naive implementation. There is no direct quantum analog of `XOR` gate, so we will use the _bit flip_ it simply changes $| 0 \rangle$ to $| 1 \rangle$ and $| 1 \rangle$ to $| 0 \rangle$.

In [48]:
import cirq

message_length = 8
alice_qubits = cirq.NamedQubit.range(message_length, prefix="alice_")
quantum_one_pad_encoder = cirq.Circuit()
for idx in range(len(secret_key_known_only_to_alice_and_bob)):
    # Prepare qubits in appropriate state
    if alice_message[idx] == 1:
        quantum_one_pad_encoder.append(cirq.X(alice_qubits[idx]))
    # Encoding message based on secret key
    if secret_key_known_only_to_alice_and_bob[idx] == 1:
        quantum_one_pad_encoder.append(cirq.X(alice_qubits[idx]))

quantum_one_pad_encoder.append(cirq.measure(alice_qubits, key="encoded_message"))

simulator = cirq.Simulator()

encryption_result = simulator.simulate(quantum_one_pad_encoder)
encoded_message = encryption_result.measurements['encoded_message']

bob_qubits = cirq.NamedQubit.range(message_length, prefix="bob_")
quantum_one_pad_decoder = cirq.Circuit()
for idx in range(len(secret_key_known_only_to_alice_and_bob)):
    if encoded_message[idx] == 1:
        quantum_one_pad_decoder.append(cirq.X(bob_qubits[idx]))
    # Decoding message based on secret key
    if secret_key_known_only_to_alice_and_bob[idx] == 1:
        quantum_one_pad_decoder.append(cirq.X(bob_qubits[idx]))

quantum_one_pad_decoder.append(cirq.measure(bob_qubits, key="decoded_message"))
decryption_result = simulator.simulate(quantum_one_pad_decoder)
decoded_message = decryption_result.measurements['decoded_message']

print("Bob decoded message:", decoded_message, "Are Alice's and Bob's messages equal?", np.all(alice_message == decoded_message))


Bob decoded message: [1 0 1 0 1 1 0 1] Are Alice's and Bob's messages equal? True


That overall looks simple, but let's say Alice want to use QC to its maximal potential and send qubit states in different base, for example  $| + \rangle$ and  $| - \rangle$. Let's update example above to see if simply changing:
```python
quantum_one_pad_encoder.append(cirq.X(alice_qubits[idx]))
```
to
```python
quantum_one_pad_encoder.append(cirq.X(alice_qubits[idx]))
# Use Hadamard base
quantum_one_pad_encoder.append(cirq.H(alice_qubits[idx]))
```
will help.

In [49]:
message_length = 8
alice_qubits = cirq.NamedQubit.range(message_length, prefix="alice_")
quantum_one_pad_encoder = cirq.Circuit()
for idx in range(len(secret_key_known_only_to_alice_and_bob)):
    # Prepare qubits in appropriate state
    if alice_message[idx] == 1:
        quantum_one_pad_encoder.append(cirq.X(alice_qubits[idx]))
        # Use Hadamard base
        quantum_one_pad_encoder.append(cirq.H(alice_qubits[idx]))
    # Encoding message based on secret key
    if secret_key_known_only_to_alice_and_bob[idx] == 1:
        quantum_one_pad_encoder.append(cirq.X(alice_qubits[idx]))

quantum_one_pad_encoder.append(cirq.measure(alice_qubits, key="encoded_message"))

simulator = cirq.Simulator()

encryption_result = simulator.simulate(quantum_one_pad_encoder)
encoded_message = encryption_result.measurements['encoded_message']

bob_qubits = cirq.NamedQubit.range(message_length, prefix="bob_")
quantum_one_pad_decoder = cirq.Circuit()
for idx in range(len(secret_key_known_only_to_alice_and_bob)):
    if encoded_message[idx] == 1:
        quantum_one_pad_decoder.append(cirq.X(bob_qubits[idx]))
    # Decoding message based on secret key
    if secret_key_known_only_to_alice_and_bob[idx] == 1:
        quantum_one_pad_decoder.append(cirq.X(bob_qubits[idx]))

quantum_one_pad_decoder.append(cirq.measure(bob_qubits, key="decoded_message"))
decryption_result = simulator.simulate(quantum_one_pad_decoder)
decoded_message = decryption_result.measurements['decoded_message']

print(
    "Alice message:      ", alice_message, "\n"
    "Bob decoded message:", decoded_message, "\n"
    "Are Alice's and Bob's messages equal?", np.all(alice_message == decoded_message)
)


Alice message:       [1 0 1 0 1 1 0 1] 
Bob decoded message: [1 0 1 0 0 1 0 1] 
Are Alice's and Bob's messages equal? False


It turns out that is not the case - and if we think a little about it, it is relatively obvious why. If we apply H-gate once and then measure qubit's state we have 50/50 chance between `0` and `1`.

We will need to apply H-gate on the decoding side also.

Before diving even deeper, we can see that we can actually use 4 qubit states namely $| 0 \rangle$, $| 1 \rangle$, $| + \rangle$, $| - \rangle$. This will make our message "denser", _but_ we will need to have twice as long key to be able to decrypt the message. In this case there is inherent tradeoff between number of qubits and bits used.
Another important difference is that now we will use quantum channel to communicate (basically, we do not measure until decryption).

In [54]:
message_length = 4
key_length = message_length * 2
short_alice_message = alice_message[:message_length]

channel_qubits = cirq.NamedQubit.range(message_length, prefix="channel_")

qubit_preparation_ops = []
qubit_encryption_ops_x = []
qubit_encryption_ops_h = []
for idx in range(len(secret_key_known_only_to_alice_and_bob) // 2):
    # Prepare qubits in appropriate state
    if short_alice_message[idx] == 1:
         qubit_preparation_ops.append(cirq.X(channel_qubits[idx]))
    # Encoding message based on secret key
    if secret_key_known_only_to_alice_and_bob[2 * idx] == 1:
        qubit_encryption_ops_x.append(cirq.X(channel_qubits[idx]))
    if secret_key_known_only_to_alice_and_bob[2 * idx + 1] == 1:
        qubit_encryption_ops_h.append(cirq.H(channel_qubits[idx]))
        
qubit_preparation_moment = cirq.Moment(qubit_preparation_ops)
qubit_encryption_moment_x = cirq.Moment(qubit_encryption_ops_x)
qubit_encryption_moment_h = cirq.Moment(qubit_encryption_ops_h)
qubit_decryption_ops_x = []
qubit_decryption_ops_h = []

for idx in range(len(secret_key_known_only_to_alice_and_bob) // 2):
    # Decoding message based on secret key. We need to apply gates in reverse order!
    if secret_key_known_only_to_alice_and_bob[2 * idx + 1] == 1:
        qubit_decryption_ops_x.append(cirq.H(channel_qubits[idx]))
    if secret_key_known_only_to_alice_and_bob[2 * idx] == 1:
        qubit_decryption_ops_h.append(cirq.X(channel_qubits[idx]))

qubit_decryption_moment_h = cirq.Moment(qubit_decryption_ops_h)
qubit_decryption_moment_x = cirq.Moment(qubit_decryption_ops_x)

quantum_channel = cirq.Circuit(
    qubit_preparation_moment, 
    qubit_encryption_moment_x, 
    qubit_encryption_moment_h, 
    qubit_decryption_moment_h,
    qubit_decryption_moment_x
)
quantum_channel.append(cirq.measure(channel_qubits, key="decoded_message"))
decryption_result = simulator.simulate(quantum_channel)
decoded_message = decryption_result.measurements['decoded_message']
print(quantum_channel)
print(
    "Alice message:      ", short_alice_message, "\n"
    "Bob decoded message:", decoded_message, "\n"
    "Are Alice's and Bob's messages equal?", np.all(short_alice_message == decoded_message)
)

channel_0: ───X───X───H───X───H───M('decoded_message')───
                                  │
channel_1: ───────X───────X───────M──────────────────────
                                  │
channel_2: ───X───X───────X───────M──────────────────────
                                  │
channel_3: ───────X───────X───────M──────────────────────
Alice message:       [1 0 1 0] 
Bob decoded message: [0 0 1 0] 
Are Alice's and Bob's messages equal? False
